## Target- urgent withdrawn by nurse(within 24hr)
* shift_applications status == 'cancelled' + shift_application prevStatus == 'confirmed'


### IDs -> only look at shifts application, leave each shifts after
* shift_applications id
* shift_applications user_id
* shift_applications shift_id


### Numeric Field
* shifts rate
* shifts net_pay: rate - nursedash profit
* shift_applications distance
* shift break time
* user withdrawn times: 
    Problem: 
        1. We don't know in reality that you will withdraw your previous one.
        2. cannot be used to predict new user
* user applied times:
* prev SA/CW rate


## Time
### shift, shift app time
* S_Create (shift created) => SA_Create (application created) => CWTime (comfim withdrawn) => Start_Time (shift start)
* S_Create: shift create time
* SA_Create: shift application create time
* CWTime: shift withdrawn time, whole confirmed withdrawn population (include within 24hr and > 24hr)
* Start_Time: shift start time
##### calculated field, graph in hours
* S_Create2SA_Create: shift application create time - shift create time
* S_Create2Start_Time: shift start time - shift create time
        1. has negative values. delete maybe?
* SA_Create2Start_Time: shift start time - shift application create time
        1. has negative values. delete maybe?

### User
* user approvedAt:
* user createdAt:
#### Calculated field
* U_Create2U_approved: User approve time - User create time
* U_Create2now: now - User create time, in months(seconds/2629746)
* U_Approve2now: now - User approve time


### Categorical
* shifts role: position name + type
* facilities name = facilities short_name
* facilities areaId = facilities areaName
* withdrawnInfo_key, withdrawnInfo_value
* facilities segmentName: Senior Living = 1, Healthcare = 0
* Users enableNotifications:
* Users emailNotifications:
* Users appNotifications:
* Users allowedNotifications:


### ?
* facilities allowedQualification
* facilities createdAt
* facilities rates
* nurse shift withdrawn by admin, when policy start, how many withdrawn by admin
* shifts = unit
* role
* users updateAt ?
* users relationToFacility

# Connect to DB, Fetch data

In [590]:
import pandas as pd
# specify connection to database
import psycopg2
connection = psycopg2.connect(
    host="nursedash-prod.cuzi2kducsnv.us-east-1.rds.amazonaws.com",
    database="nursedash",
    user="external_analyst",
    password="uDps8APganhSLc3K2xe7NtMPq")

In [591]:
# df_test = pd.read_sql_query("""

# SELECT sa.user_id, sa.shift_id, sa.id as shift_application_id,
# sa."createdAt", 
# timezone(case when f.timezone = 'CST' then 'America/Chicago' else f.timezone end, timezone('UTC', s.start_time)) as Start_Time, 
# case when f.timezone = 'CST' then 'America/Chicago' else f.timezone end as timezone,
# sa."status", sa."prevStatus",
# sa.distance, s."description" as "shift_description",
# f.id as facility_id, f.name as facility_name,
# s.net_pay, s.type
# FROM shifts s
# INNER JOIN facilities f on s.facility_id = f.id
# INNER JOIN shift_applications sa on s.id = sa.shift_id

# """, con = connection)

In [592]:
df_test = pd.read_sql_query("""

SELECT  sa.id as shift_application_id, sa.user_id, sa.shift_id, f.id AS facility_id,
sa."status", sa."prevStatus", sa."distance", s."facility_id", "s"."description" AS "shift_description",
"s"."assigned_nurse_id", s."net_pay", "s"."unit" AS "s_unit",s."type",
"s"."qualifications" AS "s_qualifications", "s"."breakTime" AS "s_breakTime",
"f"."name" AS "f_name","f"."short_name" AS "f_short_name",
timezone(case when f.timezone = 'CST' then 'America/Chicago' else f.timezone end, s."createdAt") as s_create,
timezone(case when f.timezone = 'CST' then 'America/Chicago' else f.timezone end, sa."createdAt") as sa_create,
timezone(case when f.timezone = 'CST' then 'America/Chicago' else f.timezone end, u."approvedAt") as u_approve,
timezone(case when f.timezone = 'CST' then 'America/Chicago' else f.timezone end, u."createdAt") as u_create,
timezone(case when f.timezone = 'CST' then 'America/Chicago' else f.timezone end, timezone('UTC', s.start_time)) as Start_Time 
FROM shifts s
INNER JOIN shift_applications sa ON s.id = sa.shift_id
INNER JOIN facilities f ON s.facility_id = f.id
INNER JOIN users u ON sa.user_id = u.id

""", con = connection)

In [593]:
df_test.head()

,shift_application_id,user_id,shift_id,facility_id,status,prevStatus,distance,facility_id,shift_description,assigned_nurse_id,...,type,s_qualifications,s_breakTime,f_name,f_short_name,s_create,sa_create,u_approve,u_create,start_time
0,80,4,1,1,applied,None,NaN,1,Seeking ICU specialty RN's for per-diem and lo...,2.0,...,None,None,0.0,Nursedash (Internal),ND,2018-08-02 09:25:56.428,2018-08-02 09:25:56.624,2020-03-04 12:04:44.254,2018-08-02 09:25:56.300,2017-09-12 07:00:00
1,123,3,1,1,applied,None,NaN,1,Seeking ICU specialty RN's for per-diem and lo...,2.0,...,None,None,0.0,Nursedash (Internal),ND,2018-08-02 09:25:56.428,2018-08-02 09:25:56.624,NaT,2018-08-02 09:25:56.300,2017-09-12 07:00:00
2,46,2,1,1,confirmed,None,NaN,1,Seeking ICU specialty RN's for per-diem and lo...,2.0,...,None,None,0.0,Nursedash (Internal),ND,2018-08-02 09:25:56.428,2018-08-02 09:25:56.624,NaT,2018-08-02 09:25:56.300,2017-09-12 07:00:00
3,69,2,2,1,applied,None,NaN,1,Seeking Medical Surgical specialty RN's for pe...,5.0,...,None,None,0.0,Nursedash (Internal),ND,2018-08-02 09:25:56.428,2018-08-02 09:25:56.624,NaT,2018-08-02 09:25:56.300,2017-09-17 15:00:00
4,79,4,2,1,applied,None,NaN,1,Seeking Medical Surgical specialty RN's for pe...,5.0,...,None,None,0.0,Nursedash (Internal),ND,2018-08-02 09:25:56.428,2018-08-02 09:25:56.624,2020-03-04 12:04:44.254,2018-08-02 09:25:56.300,2017-09-17 15:00:00


In [574]:
timedelta.total_seconds(df_test['u_approve'][0] - df_test['u_create'][0])/3600

7323.284045277778

In [584]:
from datetime import datetime
timedelta.total_seconds(datetime.now() - df_test['u_create'][0])/2629746

31.975228798533397

In [585]:
from datetime import timedelta
timedelta.total_seconds(datetime.now() - df_test['u_approve'][0])/2629746

21.950037519774153

In [587]:
timedelta.total_seconds(df_test['sa_create'][0] - df_test['s_create'][0])/3600

4.654403068611111

In [588]:
timedelta.total_seconds(df_test['start_time'][0] - df_test['sa_create'][0])/3600

89.01341621750001

In [549]:
'S_create2SA_Create', 'SA_Create2Start_Time', 'S_Create2Start_Time', 'CW_Time2Start_Time',
'SA_Create2CW_Time', 'S_Create2CW_Time',
'U_create2now', 'U_approve2now', 'U_create2U_approve'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172593 entries, 0 to 172592
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   user_id               172593 non-null  int64         
 1   shift_id              172593 non-null  int64         
 2   shift_application_id  172593 non-null  int64         
 3   sa_create             172593 non-null  datetime64[ns]
 4   u_approve             172015 non-null  datetime64[ns]
 5   u_create              172593 non-null  datetime64[ns]
 6   start_time            172593 non-null  object        
dtypes: datetime64[ns](3), int64(3), object(1)
memory usage: 9.2+ MB


In [438]:
pd.read_sql_query("""

SELECT "sa"."id" AS "sa.id","sa"."user_id" AS "sa.user_id","sa"."shift_id" AS "sa.shift_id","sa"."status" AS "sa.status","sa"."createdAt" AS "sa.createdAt",
"sa"."hasBreakTime" AS "sa.hasBreakTime","sa"."prevStatus" AS "sa.prevStatus","sa"."distance" AS "sa.distance",
"s1"."facility_id" AS "s1.facility_id","s1"."description" AS "s1.description","s1"."start_time" AS "s1.start_time","s1"."assigned_nurse_id" AS "s1.assigned_nurse_id",
"s1"."rate" AS "s1.rate","s1"."net_pay" AS "s1.net_pay","s1"."unit" AS "s1.unit","s1"."type" AS "s1.type","s1"."createdAt" AS "s1.createdAt","s1"."qualifications" AS "s1.qualifications",
"s1"."breakTime" AS "s1.breakTime","s1"."prevStatus" AS "s1.prevStatus","f"."id" AS "f.id","f"."name" AS "f.name","f"."short_name" AS "f.short_name"
FROM "public"."shift_applications" AS "sa"
LEFT JOIN "public"."shifts" AS "s1" ON "sa"."shift_id" = "s1"."id"
LEFT JOIN "public"."facilities" AS "f" ON "s1"."facility_id" = "f"."id"
LEFT JOIN "public"."users" AS "u" ON "sa"."user_id" = "u"."id"

""", con = connection)

# Data Cleaning

In [350]:
df = pd.read_csv('main.csv').drop(columns = ['Unnamed: 0'])
df.columns

Index(['id', 'user_id', 'shift_id', 'status', 'createdAt',
       'hasNurseCheckEvent', 'hasBreakTime', 'prevStatus', 'distance', 'id.1',
       'facility_id', 'description', 'start_time', 'assigned_nurse_id', 'rate',
       'net_pay', 'unit', 'type', 'createdAt.1', 'qualifications', 'breakTime',
       'prevStatus.1', 'id.2', 'name', 'short_name', 'createdAt.2', 'rates',
       'S_Create', 'SA_Create', 'S_create2SA_Create', 'Start_Time',
       'SA_Create2Start_Time', 'S_Create2Start_Time', 'CW_Time',
       'CW_Time2Start_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time',
       'allowedQualifications', 'areaId', 'areaName', 'segmentName',
       'U_create2now', 'U_approve2now', 'U_create2U_approve',
       'withdrawnInfo_key', 'withdrawnInfo_value'],
      dtype='object')

In [351]:
# fill these column with 0, no cw make these column nan
df[['CW_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time']] = df[['CW_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time']].fillna(0)

# There are some negative numbers in these column. Target all = 0, drop for now
df = df[(df["S_Create2Start_Time"]>0) & (df["SA_Create2Start_Time"]>0)]

# sort by time that shift was created
df = df.sort_values(by = 'createdAt').reset_index(drop = True)

In [352]:
# what are prevstatus column like when status = confirmed
df[df['status'] == 'confirmed']['prevStatus'].value_counts()

selected       34689
unavailable       10
applied            1
cancelled          1
Name: prevStatus, dtype: int64

In [353]:
# what are status column like when prevstatus = confirmed
df[df['prevStatus'] == 'confirmed']['status'].value_counts()

withdrawn    25020
cancelled     6231
applied          1
Name: status, dtype: int64

# Feature Engineering

### Target: urgent withdrawn as 1

In [354]:
def CW_by_nurse(row):
    if row['status']=='withdrawn' and row['prevStatus'] == 'confirmed':
        if row['withdrawnInfo_value'] == 'nurse':
            if row['CW_Time2Start_Time'] < 0 and row['CW_Time2Start_Time'] >= -24:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0
    
df['target'] = df.apply (lambda row: CW_by_nurse(row), axis=1)

In [355]:
df['target'].value_counts()

0    161819
1      4338
Name: target, dtype: int64

###  Count previsous shift application

In [356]:
# track by using dictionary
count_prev_SA = []
test = df[['user_id', 'createdAt']]

# create SA dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
sa_dict = {} 
for uid in uid_library:
    sa_dict.update({uid: 0}) 
    
for i, v in enumerate(test['user_id']):
    sa_dict[v] += 1
    count_prev_SA.append(sa_dict[v]-1)
 
df['count_prev_SA'] = count_prev_SA

###  Count previsous urgent withdrawns

In [357]:
# track by using dictionary
count_prev_CW = []
test = df[['user_id', 'count_prev_SA', 'createdAt', 'target']]

# create cw dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
cw_dict = {} 
for uid in uid_library:
    cw_dict.update({uid: 0}) 
    
# fill dictionary and fill cw count
for i, v in enumerate(test['user_id']):
    if test['target'][i] == 1:
        cw_dict[v] += 1
        count_prev_CW.append(cw_dict[v]-1)
    else:
        count_prev_CW.append(cw_dict[v])
        
df['count_prev_CW'] = count_prev_CW

### Count previsous urgent withdrawns/Count previsous shift applications

In [358]:
df['prev_CW/SA_rate'] = df['count_prev_CW']/df['count_prev_SA']

# fill nan with 0, happend bc 0/0, meaning rate = 0
df['prev_CW/SA_rate'].fillna(0,inplace=True)

### previous withdrawn times previous apply

In [359]:
df['prev_CW x SA_rate'] = df['count_prev_CW']*df['count_prev_SA']

### 2 Type dummy: RN, LVN + LPN , Rest


In [360]:
df['type'].value_counts()

CNA             85316
STNA            39643
LVN             15695
RN              11689
LPN              6652
CMA/Med-Tech     6238
Tech              924
Name: type, dtype: int64

In [361]:
df['type_RN'] = df.apply(lambda row:1 if row['type'] == 'RN' else 0, axis = 1)
df['type_LVN+LPN'] = df.apply(lambda row: 1 if row['type'] == 'LVN' or row['type'] == 'LPN' else 0, axis = 1)

### 1 SegementName dummy: Senior Living, Healthcare

In [362]:
df['segmentName'].value_counts()

Senior Living    145625
Healthcare        20517
Name: segmentName, dtype: int64

In [363]:
def create_segmentName_dummy(row):
    if row['segmentName']=='Senior Living':
        return 1
    else:
        return 0
    
df['segmentName_d'] = df.apply(lambda row: create_segmentName_dummy(row), axis=1)

### 5 Facility Area Name Dummy

In [364]:
df['areaName'].value_counts()

Houston           83626
Northeast Ohio    46539
DFW               26611
Austin             5959
San Antonio        3302
Cincinnati          120
Name: areaName, dtype: int64

In [365]:
# Houston
df['areaName_houston'] = df.apply(lambda row: 1 if row['areaName'] == 'Houston' else 0, axis = 1)

# Northeast Ohio
df['areaName_no'] = df.apply(lambda row: 1 if row['areaName'] == 'Northeast Ohio' else 0, axis = 1)

# DFW
df['areaName_dfw'] = df.apply(lambda row: 1 if row['areaName'] == 'DFW' else 0, axis = 1)

# Austin
df['areaName_austin'] = df.apply(lambda row: 1 if row['areaName'] == 'Austin' else 0, axis = 1)

# San Antonio
df['areaName_san'] = df.apply(lambda row: 1 if row['areaName'] == 'San Antonio' else 0, axis = 1)

### <font color = blue>keep only prevstatus or status = confirmed

In [366]:
df = df[df.apply(lambda row: (row['prevStatus'] == 'confirmed') or (row['status'] == 'confirmed'), axis = 1)]

### <font color = green>output feature dataframe

In [367]:
df.columns

Index(['id', 'user_id', 'shift_id', 'status', 'createdAt',
       'hasNurseCheckEvent', 'hasBreakTime', 'prevStatus', 'distance', 'id.1',
       'facility_id', 'description', 'start_time', 'assigned_nurse_id', 'rate',
       'net_pay', 'unit', 'type', 'createdAt.1', 'qualifications', 'breakTime',
       'prevStatus.1', 'id.2', 'name', 'short_name', 'createdAt.2', 'rates',
       'S_Create', 'SA_Create', 'S_create2SA_Create', 'Start_Time',
       'SA_Create2Start_Time', 'S_Create2Start_Time', 'CW_Time',
       'CW_Time2Start_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time',
       'allowedQualifications', 'areaId', 'areaName', 'segmentName',
       'U_create2now', 'U_approve2now', 'U_create2U_approve',
       'withdrawnInfo_key', 'withdrawnInfo_value', 'target', 'count_prev_SA',
       'count_prev_CW', 'prev_CW/SA_rate', 'prev_CW x SA_rate', 'type_RN',
       'type_LVN+LPN', 'segmentName_d', 'areaName_houston', 'areaName_no',
       'areaName_dfw', 'areaName_austin', 'areaName_san'],
    

In [368]:
feature_df = df[['id', 'user_id', 'shift_id', 'prev_CW/SA_rate', 'status', 'S_create2SA_Create',
                 'S_Create2Start_Time',
                 'SA_Create2Start_Time','U_create2now', 'U_approve2now','prev_CW x SA_rate', 'type_RN', 
                 'type_LVN+LPN', 'segmentName_d', 'areaName_houston', 'areaName_no', 'areaName_dfw', 
                 'areaName_austin', 'areaName_san', 'net_pay', 'distance','target', 'createdAt', 'Start_Time']]
feature_df.to_csv('model_data.csv')